# <center>Traitement d'image du projet TouNum <br />Livrable 1</center>

![cesiLogo.jpg](cesiLogo.jpg)

|Auteur|Centre|Modification|
|---|---|---|
|A. RIGAUT|Arras|2020/12/15|
|B. THIBAULT|Arras|2020/12/15|
|R. VANCAMP|Arras|2020/12/15|
|T. POLY|Arras|2020/12/15|
|V. NAESSENS|Arras|2020/12/15|

</div>

## Contexte

L'entreprise TouNoum nous a chargé de traiter deux jeux d'image l'un bruité et le second flouté. Le but est de rendre les images plus traitables par un algorithme de machine learning.

## Jeux de données

Dans le dossier Dataset nous avons les deux jeux de données dans deux sous-dossier, Noisy(bruité) et Blurry(flouté).
## Explication chargement de fichier

In [86]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import os
from scipy import misc
from skimage.io import imread_collection, imsave
from skimage import *
from skimage.filters import unsharp_mask
from skimage.restoration import estimate_sigma #Robust wavelet-based estimator of the (Gaussian) noise standard deviation
import cv2


#creating a collection with the available images
noisy = imread_collection('Noisy/*.jpg')
blurry = imread_collection('Blurry/*.jpg')
noisyPost = []
blurryPost = []


## Test

Lors de nos phases de tests nous avons utilisé la librairie Scipy. Cette bibliothèque ne permet pas de traiter les images RGB (multicanaux). Il y a donc deux démarches qui se proposaient à nous, passer l'image en niveau de gris ou séparer les canaux RGB pour les traiter indépendamment.

Dans un premier temps, nous avons testé de passer d'une image RGB à une image en niveau de gris grâce à la fonction rgb2gray. Celle-ci se base sur la formule:
$gray = 0.2989 * redChannel + 0.5870 * greenChannel + 0.1140 * blueChannel$ <br />

In [87]:
def rgb2gray(image):
    """Transform a polychrome image in a grey image
    Parameters
    ----------
    image : array-like, shape (width, height, channel)
        Data
        
    Returns
    -------
    Gray like image. One channel
    """
    r, g, b = image[:,:,0], image[:,:,1], image[:,:,2] #isolate each channel
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b #more detail about this scale: https://en.wikipedia.org/wiki/Grayscale
    return gray

L'image changé en nuances de gris, nous pouvons appliquer de manière simple et rapide un ensemble de filtre que se soit des matrices ou des fonctions. Cependant, cette méthode fait perdre une caractéristique de l'image, sa couleur. En effet, on imagine assez facilement l'intérêt d'une couleur pour un algorithme de machine learning.

De se fait, nous avons testé de traiter les canaux individuellement, avec la fonction splitColor qui nous permet de séparer les trois canaux RGB. Cette fonction récupère l'intégralité des pixels d'un canal de l'image, canal qui est représenté par une dimension du tableau représentatif de l'image.

In [88]:
def splitColor(image):
    """Create a tiple witch each channel
    Parameters
    ----------
    image : array-like, shape (width, height, channel)
        Data
        
    Returns
    -------
    Tuple (red channel, green channel, blue channel)
    """
    red = image[:, :, 0] #all x and y in channel 0 whose is red
    green = image[:, :, 1]
    blue = image[:, :, 2]
    return(red, green, blue)

Puis chacun de ces channels ont été traités dans la fonction noisyTreatment. 

En possession des différents canaux, nous pouvons appliquer nos filtres sur chacun d'entre eux. Ici, nous appliquons un filtre de Gauss à un canal de notre image d'origine afin de la lisser dans l'objectif de réduire le bruit. Suite à ce traitement, nous appliquons la matrice:
$$\begin{bmatrix} -1 & -1 & -1 \\ -1 & 9 & -1 \\ -1 & -1 & -1 \end{bmatrix}$$
Celle-ci est l'addition de deux matrices: la matrice identité et la matrice de détection de contour :
$$\begin{bmatrix} 0 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 0 \end{bmatrix} + \begin{bmatrix} -1 & -1 & -1 \\ -1 & 8 & -1 \\ -1 & -1 & -1 \end{bmatrix} = \begin{bmatrix} -1 & -1 & -1 \\ -1 & 9 & -1 \\ -1 & -1 & -1 \end{bmatrix}$$

In [89]:
def noisyTreatment(channel):
    """Create a tuple witch each channel
    Parameters
    ----------
    channel : array-like, shape (width, height, channel)
        Data
        
    Returns
    -------
    array-like, shape (width, height)
    """
    arr = []
    
    filter0 = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    for layer in layers:
        gauss_denoised = ndimage.gaussian_filter(channel, sigma=2.5)
        channelFlou = scipy.ndimage.convolve(gauss_denoised, filter0)
        result = channel + (channel - channelFlou) * 1
        arr.append(result)

    return np.dstack((arr[0], arr[1], arr[2]))

Puis afin de standardiser nos valeurs, nous décidons de redistribuer nos valeurs entre 0 et 255.

In [90]:
def singleChannel(image, sig, vrange):
    """Perform action in a single channel of an image
    Parameters
    ----------
    image : array-like, shape (width, height, channel)
        Data
    sig : int
        Standard deviation
    vrange : array-like, shape (minus, plus)
        
    Returns
    -------
    Image processed
    """
    
    blurred = ndimage.gaussian_filter(image, sigma=sig, mode='reflect') #apply gaussian filter
    result = image + (image - blurred) * 1
    if vrange is not None:
        return np.clip(result, vrange[0], vrange[1], out=result) #Given an interval, values outside the interval are clipped to the interval edges. For example, if an interval of [0, 1] is specified, values smaller than 0 become 0, and values larger than 1 become 1.
    return result

Tout comme la phase de test précédente les résultats ne nous convenaient pas, car certaines valeurs étaient aberrantes.

## Solution

### Explication

Dans cette partie, nous avons utilisé la bibliothèque Scikit-image.

### Processus

Pour résoudre les problèmes de flou et de bruit, nous utilisons la fonction unsharp_mask de skimage.filters.
Celle-ci réalise deux processus:
<ol>
<li>Elle applique un filtre gaussien afin de lisser l'image. </li>
<li>Puis elle applique la même matrice que précédemment afin d'améliorer la netteté de l'image $$\begin{bmatrix} -1 & -1 & -1 \\ -1 & 9 & -1 \\ -1 & -1 & -1 \end{bmatrix}$$</li>
</ol>


Dans un second temps la formule suivante est appliquée : 

$$enhanced Image = original + amount * (original - blurred)$$

$amount$ correspond à l'amplification des détails de l'image, plus celui-ci est élevé, plus les contrastes sont élevés.

In [93]:
def process_noisy(collection):
    """Process noisy image
    Parameters
    ----------
    collection : array-like, shape (file)
        Data
        
    Returns
    -------
    """
    for file in range(len(collection)):
        result = unsharp_mask(collection[file], radius=1, amount=1)
        noisyPost.append(result)
        imsave('noisyEnd/' + collection.files[file][-7:-4] + '.jpg', img_as_ubyte(result))
    return 1
        
def process_blurry(collection):
    """Process blurry image
    Parameters
    ----------
    collection : array-like, shape (file)
        Data
        
    Returns
    -------
    """
    for file in range(len(collection)):
        result = unsharp_mask(collection[file], radius=0, amount=1)
        blurryPost.append(result)
        imsave('blurryEnd/' + collection.files[file][-7:-4] + '.jpg', img_as_ubyte(result))
    return 1

In [97]:
#process_noisy(noisy)
#process_blurry(blurry)

## Indice de performance

### Images bruitées

Dans cette partie de notre traitement, le but est de prouver l'efficacité de notre traitement d'image.

Pour se faire nous avons utilisé la fonction estimate_sigma() de la biliothèque scikit-image. 

Cette fonction permet d'estimer l'écart-type de la fonction de Gauss. Puis nous faisons une moyenne de tous les écarts types des images de base et une moyenne des écarts type des images traitées. Enfin, elle détermine le taux de variation entre les deux moyennes obtenues. La fonction nous renvoie un résultat avoisinant les 40%, ce qui signifie que la fonction pour atténuer le bruit est performante.

In [81]:
# Get the Denoised images
noisyTreated = imread_collection('noisyEnd/*.jpg')

def estimate_noise(listNoise, listDeNoise):
    """Calculates noise rates in an image
    Parameters
    ----------
    listNoise : array-like, shape (file)
        Data
    listDeNoise: array-like, shape (file)
        Data
        
    Returns
    -------
    Sigma is the mean rate of noise
        
    """
    listSigmaNoise = []
    listSigmaDenoise = []
    # Get the noise sigma of each image then add it to the list
    for imageNoise in listNoise:
        sigmaNoise = estimate_sigma(imageNoise, multichannel=True, average_sigmas=True)
        listSigmaNoise.append(sigmaNoise)
    
    # Calculation of the mean sigma of the noised images
    totalSigmaNoise = sum(listSigmaNoise)
    meanSigmaNoise = totalSigmaNoise/len(listNoise)
    
    # Same action here for denoised images
    for imageDenoise in listDeNoise:
        sigmaDenoise = estimate_sigma(imageDenoise, multichannel=True, average_sigmas=True)
        listSigmaDenoise.append(sigmaDenoise)

    totalSigmaDenoise = sum(listSigmaDenoise)
    meanSigmaDenoise = totalSigmaDenoise/len(listDeNoise)

    return(meanSigmaNoise,meanSigmaDenoise)

# Calculation of the percentage difference between the sigma to obtain the performance index
meanSigmaNoise,meanSigmaDenoise = estimate_noise(noisy,noisyTreated)
performance_index_noisy = ((meanSigmaDenoise - meanSigmaNoise)/meanSigmaNoise)*100
print(round(performance_index_noisy,2))

39.91


### Images floutées

Pour connaître la performance de notre processus, nous avons dû détecter le flou dans notre image originale. Pour cela, nous utilisons la variance de Laplace grâce à « cv2.Laplacian(collectionPre[file], cv2.CV_64F).var() ». Plus le résultat est proche de 0, plus l’image est floutée. Donc nous l’appliquons sur notre image de base puis sur notre image après traitement et nous ajoutons ces données dans un dataframe pour que cela soit plus lisible et exploitable par la suite. Comme vous pouvez le voir le résultat de la fonction appliquée aux images post-traitement est proche de zéro, ce qui montre que le traitement est un succès.

In [95]:
def blurryDetection(collectionPre, collectionPost): 
    """Calculates blur rate in an image
    Parameters
    ----------
    collectionPre : array-like, shape (file)
        Data
    collectionPost: array-like, shape (file)
        Data
        
    Returns
    -------
    Dataframe with the comparison of detection before and after treatment
        
    """
    Df= pd.DataFrame(columns=['FileName','Blur rate pre-treatment','Blur rate post-treatment'])
    for file in range(len(collectionPre)):
        imagePre = cv2.Laplacian(collectionPre[file], cv2.CV_64F).var()
        imagePost = cv2.Laplacian(collectionPost[file], cv2.CV_64F).var()
        Df=Df.append({'FileName':collectionPre.files[file],'Blur rate pre-treatment':imagePre,'Blur rate post-treatment':imagePost},ignore_index=True)
    print("Max_traitement : ","\n", Df[ Df['Blur rate post-treatment'] == Df['Blur rate post-treatment'].max() ],"\n")
    print("Min_traitement : ","\n",Df[ Df['Blur rate post-treatment'] == Df['Blur rate post-treatment'].min() ],"\n")
    print("Variance_After_traitement : ", Df['Blur rate post-treatment'].var() ,"\n")
    print("Mean_After_traitement : ", Df['Blur rate post-treatment'].mean() ,"\n")
    return Df

In [96]:
blurryDetection(blurry, blurryPost)

,FileName,Blur rate pre-treatment,Blur rate post-treatment
0,Blurry\blurry_001.jpg,417.029056,0.006413
1,Blurry\blurry_002.jpg,65.037823,0.001000
2,Blurry\blurry_003.jpg,607.788792,0.009347
3,Blurry\blurry_004.jpg,245.151501,0.003770
4,Blurry\blurry_005.jpg,227.685810,0.003502
...,...,...,...
145,Blurry\blurry_146.jpg,137.893416,0.002121
146,Blurry\blurry_147.jpg,262.208983,0.004032
147,Blurry\blurry_148.jpg,89.037813,0.001369
148,Blurry\blurry_149.jpg,112.238551,0.001726
